## An exploration of piq and HINT footprints on chr21

I reported that our piq footprints (for 18 samples and ~550 motifs) totaled > 2B, but HINT footprints totaled only about 20M: two orders of magnitude.   Seth helpfully explained that we should expect to see a pretty good match of HINT and piq results if we filtered the piq results on some one or more of the scores piq reports.  I proposed that I do some exploratory (visual) data analysis of this matter, asked Seth for some thresholds.   Seth's reply:

<code>
Two approaches seem reasonable for filtering the PIQ results to get a smaller number of high-confidence footprints.

1. The authors of PIQ recommend performing filtering based on the purity score. "Purity" is an odd term for 1 - FDR. i.e., a purity score of 0.7 corresponds to a 30% (predicted) False Discovery Rate. I think this is fine, but I'm not sure I trust their purity score will really function as an FDR...

2. Alternatively, we can use the comparison to ChIP-seq to guide us.  It is common to define a cutoff value for the predictor that achieves a balance between precision and recall (i.e., the "precision-recall break even" point). Note that we can apply this approach to scores from Wellington and HINT, as well as PIQ. I calculated the PRBE for each of the three footprinting algorithms. For PIQ, I used the "score" column in my analysis, since this combines evidence about both the motif sequence and the shape of the DNase. The PRBE point will differ between TFs, so we'll have to see if there is a reasonable value to use across all the TFs with ChIP-seq. Here are the PRBE values for ELF1:

         AUROC              PRBE        Precision.PRBE
fimo    0.6566       14.62802959     0.124315387000103
well.   0.8938       19.14902115     0.591500347532519
hint    0.9742       228.1427216     0.710662422916252
piq     0.7833       40.51518076     0.5462218250422
</code>

This notebook
   * makes connections to temporary databases (chr21 only) for HINT and piq
   * gets some summary statistics
   * displays comparision igv tracks conditioned on region and scores
   

In [1]:
import pandas as pd, psycopg2 as psql
from igv import IGV, Reference, Track
import time

### make connections to the (currently separate) databases.  they wil be merged soon.

In [2]:
dbHint = psql.connect("dbname=hintTest2  user=pshannon")

In [3]:
dbpiq = psql.connect("dbname=piqTest user=pshannon")

### each database has two tables: regions, and hits.  learn their sizes before filtering

In [4]:
regionCount = pd.read_sql_query("select count(*) from regions", dbHint).ix[0,0]
hitCount = pd.read_sql_query("select count(*) from hits", dbHint).ix[0,0]
print("HINT: %d hits in %d regions, density %4.2f" % (hitCount, regionCount, 
                                                      hitCount/regionCount))

HINT: 504863 hits in 222660 regions, density 2.27


In [5]:
regionCount = pd.read_sql_query("select count(*) from regions", dbpiq).ix[0,0]
hitCount = pd.read_sql_query("select count(*) from hits", dbpiq).ix[0,0]
print("piq: %d hits in %d regions, density %4.2f" % (hitCount, regionCount, 
                                                     hitCount/regionCount))

piq: 30072096 hits in 1158121 regions, density 25.97


### An arbitrary choice: look upstream from the tss of the APP gene

In [6]:
APP_tss = 25880550
loc_chrom = "chr21"
loc_start = APP_tss - 3000
loc_stop  = APP_tss + 0

### define a function which encapsulates the sql join, returns a table in bed format

In [7]:
def getHits(db, chrom, start, end):
  query = """select * from regions r
    inner join hits h on r.loc = h.loc
    where r.chrom = '%s' and r.start > %d and r.stop < %d""" % (chrom, start, end)
  tbl = pd.read_sql_query(query, db)
  return(tbl[["chrom", "start", "stop", "name", "score1", "strand", "score2", "score3", "score4"]])


In [8]:
tbl_hint = getHits(dbHint, "chr21", APP_tss-2000, APP_tss+100)
tbl_hint

,chrom,start,stop,name,score1,strand,score2,score3,score4
0,chr21,25879515,25879523,MA0597.1,39.0,-,11.4474,0.000008,None
1,chr21,25879510,25879519,MA0620.1,39.0,+,10.4082,0.000094,None
2,chr21,25879513,25879523,MA0646.1,39.0,+,10.3673,0.000086,None
3,chr21,25879839,25879847,MA0738.1,13.0,+,11.5273,0.000030,None


In [9]:
tbl_piq = getHits(dbpiq, "chr21", APP_tss-2000, APP_tss+100)

In [10]:
tbl_piq.shape

(2034, 9)

In [12]:
tbl_piq.gt07 = tbl_piq[tbl_piq["score4"] > 0.7]
tbl_piq.gt07.shape

(88, 9)

### create an igv track viewer with gene reference track, then one each for piq & hint

In [14]:
igv = IGV(locus="chr21:25,863,045-25,887,052", reference=Reference(id="hg38"), 
          tracks=[Track(
                   name="Genes hg38 v24",
                   format="gtf",
                   url="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz",
                   indexURL="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz.tbi",
                   visibility_window=10000000,
                   display_mode="SQUISHED")])


In [15]:
igv

In [16]:
def createTrack(tblHits, trackName):
   tblHits.to_csv("%s.bed" % trackName, sep="\t", header=False, index=False)
   print("wrote %s.bed" % trackName)
   newTrack = Track(name=trackName, 
                 format="bed", 
                 indexed=False, 
                 url="http://whovian:9999/files/%s.bed" % trackName, 
                 display_mode='EXPANDED')
   return(newTrack)
   

In [17]:
x = createTrack(tbl_piq, "piq.gt.07")

wrote piq.gt.07.bed


In [ ]:
igv.load_track(x)

In [ ]:
x2 = createTrack(tbl_hint, "HINT")
igv.load_track(x2)